In [1]:
import numpy as np
import pandas as pd

In [2]:
folder = "../data/eval_data/"

path_list = []
for i in range(1, 23):
    filename = f"gaze_user{i}_eval.csv"
    path_list.append(folder+filename)

path_list

['../data/eval_data/gaze_user1_eval.csv',
 '../data/eval_data/gaze_user2_eval.csv',
 '../data/eval_data/gaze_user3_eval.csv',
 '../data/eval_data/gaze_user4_eval.csv',
 '../data/eval_data/gaze_user5_eval.csv',
 '../data/eval_data/gaze_user6_eval.csv',
 '../data/eval_data/gaze_user7_eval.csv',
 '../data/eval_data/gaze_user8_eval.csv',
 '../data/eval_data/gaze_user9_eval.csv',
 '../data/eval_data/gaze_user10_eval.csv',
 '../data/eval_data/gaze_user11_eval.csv',
 '../data/eval_data/gaze_user12_eval.csv',
 '../data/eval_data/gaze_user13_eval.csv',
 '../data/eval_data/gaze_user14_eval.csv',
 '../data/eval_data/gaze_user15_eval.csv',
 '../data/eval_data/gaze_user16_eval.csv',
 '../data/eval_data/gaze_user17_eval.csv',
 '../data/eval_data/gaze_user18_eval.csv',
 '../data/eval_data/gaze_user19_eval.csv',
 '../data/eval_data/gaze_user20_eval.csv',
 '../data/eval_data/gaze_user21_eval.csv',
 '../data/eval_data/gaze_user22_eval.csv']

In [3]:
s = 0.0875
t = 0.071

marker_pos = np.array([[-2*s, -2*s-t],
                   [-s, -2*s-t],
                   [0.0, -2*s-t],
                   [s, -2*s-t],
                   [2*s, -2*s-t],
                   [-2*s, -s-t],
                   [-s, -s-t],
                   [0.0, -s-t],
                   [s, -s-t],
                   [2*s, -s-t],
                   [-2*s, 0.0-t],
                   [-s, 0.0-t],
                   [0.0, 0.0-t],
                   [s, 0.0-t],
                   [2*s, 0.0-t],
                   [-2*s, s-t],
                   [-s, s-t],
                   [0.0, s-t],
                   [s, s-t],
                   [2*s, s-t],
                   [-2*s, 2*s-t],
                   [-s, 2*s-t],
                   [0.0, 2*s-t],
                   [s, 2*s-t],
                   [2*s, 2*s-t]])

marker_pos

array([[-0.175 , -0.246 ],
       [-0.0875, -0.246 ],
       [ 0.    , -0.246 ],
       [ 0.0875, -0.246 ],
       [ 0.175 , -0.246 ],
       [-0.175 , -0.1585],
       [-0.0875, -0.1585],
       [ 0.    , -0.1585],
       [ 0.0875, -0.1585],
       [ 0.175 , -0.1585],
       [-0.175 , -0.071 ],
       [-0.0875, -0.071 ],
       [ 0.    , -0.071 ],
       [ 0.0875, -0.071 ],
       [ 0.175 , -0.071 ],
       [-0.175 ,  0.0165],
       [-0.0875,  0.0165],
       [ 0.    ,  0.0165],
       [ 0.0875,  0.0165],
       [ 0.175 ,  0.0165],
       [-0.175 ,  0.104 ],
       [-0.0875,  0.104 ],
       [ 0.    ,  0.104 ],
       [ 0.0875,  0.104 ],
       [ 0.175 ,  0.104 ]])

In [4]:
def getGazeData(gaze_data, extracted_start, extracted_stop):
    extracted_data = np.empty([0, np.shape(gaze_data)[1]])
    for i in range(0, 25):
        extracted_data = np.concatenate([extracted_data, gaze_data[extracted_start[i]:extracted_stop[i]]])
    return extracted_data

def getMarkerPos(extracted_start, extracted_stop):
    extracted_marker_pos = np.empty([0, 2])
    for i in range(0, 25):
        marker_num = flag[extracted_stop[i]]
        data_num = extracted_stop[i]- extracted_start[i]
        tmp = np.ones((data_num, 2))*marker_pos[marker_num-1]
        extracted_marker_pos = np.vstack([extracted_marker_pos, tmp])
    return extracted_marker_pos

In [5]:
for i in range(22):

    print()
    print(f"path: {path_list[i]}")
    df = pd.read_csv(path_list[i])
    print(f"df.shape: {df.shape}")
    
    # get measurement start timing
    t = 0
    while 1:
        if df["flag"][t] != 0:
            break
        t += 1
    
    flag = df["flag"].values[t:]
    time = df["time"].values[t:]
    gaze_data = df.values[t:]
    print(f"gaze_data.shape: {gaze_data.shape}")
    
    # extract gaze timing
    start = [0]
    stop = []
    for j in range(1, len(flag)-1):
        if flag[j] != 0:
            if flag[j-1] == 0:
                start.append(j)
            if flag[j+1] == 0:
                stop.append(j)
    
    # extract only the middle 1 second
    extracted_start = []
    extracted_stop = []
    for k in range(len(start)):
        extracted_start.append(int(start[k]+(stop[k]-start[k])/3))
        extracted_stop.append(int(start[k]+(stop[k]-start[k])*2/3))
    
    extracted_data = getGazeData(gaze_data, extracted_start, extracted_stop)
    extracted_marker_pos = getMarkerPos(extracted_start, extracted_stop)
    print(f"extracted_data.shape: {extracted_data.shape}")
    print(f"extracted_marker_pos.shape: {extracted_marker_pos.shape}")
    
    data = np.concatenate([extracted_data, extracted_marker_pos], axis=1)
    print(f"data.shape: {data.shape}")
    
    col = np.array(df.columns)
    col = np.append(col, ["u", "v"])
    
    # create dataframe
    df_25 = pd.DataFrame(data, columns=col)
    df_16 = df_25[df_25["flag"].isin([2, 4, 6, 7, 8, 9, 10, 12, 14, 16, 17, 18, 19, 20, 22, 24])]
    df_9 = df_25[df_25["flag"].isin([1, 3, 5, 11, 13, 15, 21, 23, 25])]
    
    # save csv file
    df_25.to_csv(folder + f"gaze_user{i+1}_eval_25.csv", index=False, encoding="utf_8_sig")
    df_16.to_csv(folder + f"gaze_user{i+1}_eval_16.csv", index=False, encoding="utf_8_sig")
    df_9.to_csv(folder + f"gaze_user{i+1}_eval_9.csv", index=False, encoding="utf_8_sig")


path: ../data/eval_data/gaze_user1_eval.csv
df.shape: (10043, 52)
gaze_data.shape: (8557, 52)
extracted_data.shape: (1570, 52)
extracted_marker_pos.shape: (1570, 2)
data.shape: (1570, 54)

path: ../data/eval_data/gaze_user2_eval.csv
df.shape: (7966, 52)
gaze_data.shape: (7253, 52)
extracted_data.shape: (1568, 52)
extracted_marker_pos.shape: (1568, 2)
data.shape: (1568, 54)

path: ../data/eval_data/gaze_user3_eval.csv
df.shape: (6806, 52)
gaze_data.shape: (6395, 52)
extracted_data.shape: (1563, 52)
extracted_marker_pos.shape: (1563, 2)
data.shape: (1563, 54)

path: ../data/eval_data/gaze_user4_eval.csv
df.shape: (6408, 52)
gaze_data.shape: (6079, 52)
extracted_data.shape: (1563, 52)
extracted_marker_pos.shape: (1563, 2)
data.shape: (1563, 54)

path: ../data/eval_data/gaze_user5_eval.csv
df.shape: (7485, 52)
gaze_data.shape: (6977, 52)
extracted_data.shape: (1558, 52)
extracted_marker_pos.shape: (1558, 2)
data.shape: (1558, 54)

path: ../data/eval_data/gaze_user6_eval.csv
df.shape: (719